In [46]:
from cleanText import text_clean, clearColumns
from inputData import textTransfer
from standardAddress import addressList, matchAddress, loadAddressList, normalizeAddress
from validAddress import ValidAddress

matchAddress = ValidAddress()

provinceName = "./data/provinceLib.csv"

standardAddressList = [text_clean(item) for item in addressList(provinceName)]
listAddress = loadAddressList(provinceName)

# replace the fail characteristic and the unexpected location words
def replaceCharList(text):
    replaceList = [".,", "-,", ",,","--",".-",",VIET NAM",",VN", ",VIETNAM",", VIET NAM",", VN", ", VIETNAM",'-VIETNAM','-VIET NAM',"VIET NAM","VN", "VIETNAM",]
    textResults = text_clean(text)
    for item in replaceList:
        textResults = textResults.replace(item," ")
    return textResults
    
def parseAddress(item):
    if (item.find(',') != -1):
        textList = item.split(",")
    elif (item.find("-") != -1):
        textList = item.split("-")
    else:
        textList = [0]
    textList = [clearColumns(value.strip()) for value in textList if clearColumns(value.strip()) != '']
    extractList = ["0","0","0"]
    lenList = len(textList)
    missedTextList = None
    if lenList < 3:
        missedItem = textTransfer(item)
        missedCharList = [",,", ', ,', '-']
        for char in missedCharList:
            missedItem = missedItem.replace(char, ",")
        missedTextList = missedItem.split(",")
        lenListTemp = len(missedTextList)
        if lenListTemp < 3:
            extractList = ["0", "0", "0"]
        else:
            extractList = [missedTextList[lenListTemp-1],
                            missedTextList[lenListTemp-2], missedTextList[lenListTemp-3]]
        for value in extractList:
            missedTextList.remove(value)
        extractList = normalizeAddress(extractList)
        return ','.join(missedTextList), clearColumns(extractList[2]) + ", "+ clearColumns(extractList[1]) + ", " + clearColumns(extractList[0])
    else:
        extractList = [textList[lenList-1],
                        textList[lenList-2], textList[lenList-3]]
        for value in extractList:
            textList.remove(value)
        extractList = normalizeAddress(extractList)
        return ','.join(textList), clearColumns(extractList[2]) + ", "+ clearColumns(extractList[1]) + ", " + clearColumns(extractList[0])
        

In [47]:
text = '33 VAN PHU - PHU LA - HA DONG - HN'
results = replaceCharList(text)
print(results)
other, results = parseAddress(results)
result = matchAddress.valid_address(results)
print(result)
print('địa chỉ khác ',other)

33 VAN PHU - PHU LA - HA DONG - HN
['HA NOI', 'HA DONG', 'PHU LA']
HA NOI
{'province': 'HA NOI', 'district': 'HA DONG', 'ward': 'PHU LA'}
địa chỉ khác  33 VAN PHU


In [48]:
f = open('datatest.txt','r')
lines = f.readlines()
for line in lines:
    print(line.strip())
    results = replaceCharList(line)
    other, results = parseAddress(results)
    result = matchAddress.valid_address(results)
    print(result)
    print('địa chỉ khác ',other)
    print('------------------------------')

01 DAO TRINH NHAT, VY DA, HUE
{'province': None, 'district': None, 'ward': None}
địa chỉ khác  
------------------------------
NGO 68 DUONG MAC DANG DUNG,SO NHA 03, PHUONG DONG MAI, THI XA QUANG YEN, TINH QUANG NINH
['QUANG NINH', 'QUANG YEN', 'DONG MAI']
QUANG NINH
{'province': 'QUANG NINH', 'district': 'QUANG YEN', 'ward': 'DONG MAI'}
địa chỉ khác  NGO 68 DUONG MAC DANG DUNG,SO NHA 03
------------------------------
TRUC LY, VO NINH, QUANG NINH, QUANG BINH
['QUANG BINH', 'QUANG NINH', 'VO NINH']
QUANG BINH
{'province': 'QUANG BINH', 'district': 'QUANG NINH', 'ward': 'VO NINH'}
địa chỉ khác  TRUC LY
------------------------------
LE XA, TIEN NU, HUNG YEN
['HUNG YEN', 'TIEN NU', 'LE XA']
HUNG YEN
{'province': 'HUNG YEN', 'district': None, 'ward': None}
địa chỉ khác  
------------------------------
69 nui doi- thi tran soc son- huyen soc son- tp ha noi
['HA NOI', 'SOC SON', 'SOC SON']
HA NOI
{'province': 'HA NOI', 'district': 'SOC SON', 'ward': 'SOC SON'}
địa chỉ khác  69 NUI DOI
-------